In [104]:
import pandas as pd
import numpy as np
from google.colab import drive
import os
import warnings
warnings.filterwarnings('ignore')
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/Alexes_Bansal_Lab/Prison Vaccinaiton Project/')

file = open('Data/Original/historical_state_counts.csv')
df_main = pd.read_csv(file, low_memory = False, encoding= 'unicode_escape')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [105]:
## Removing Unnecessary Columns found in data frame for clarity and rows with nans.
l3=['Residents.Confirmed', 'Staff.Confirmed',
       'Residents.Deaths', 'Staff.Deaths', 'Residents.Tadmin',
       'Residents.Tested', 'Residents.Active', 'Staff.Active','Staff.Initiated','Staff.Initiated.Pct', 'Residents.Completed', 'Staff.Completed',
       'Residents.Completed.Pct', 'Residents.Vadmin', 'Staff.Vadmin','Residents.Initiated.Pct']
for col in l3:
    df_main=df_main.drop(col,axis=1)

df_main=df_main.dropna(axis=0,subset=['Residents.Initiated'])

## Adding in month and year column
df_main['Date'] = pd.to_datetime(df_main.Date, format='%Y-%m-%d')
df_main['Month']=pd.DatetimeIndex(df_main['Date']).month
df_main['Year']=pd.DatetimeIndex(df_main['Date']).year


In [113]:
## Getting data on population for each state jurisdiction to serve as denominators
file2=open('Data/Original/state_aggregate_denominators.csv')
df_denom=pd.read_csv(file2)

pop_pairs=[]
for row in range(0,52):
    pop_pairs.append((df_denom['State'][row],df_denom['Residents.Population'][row]))

## Here we use the population data from December 2021 becuase at the time of analysis it was the most complete and recent data available
df_main['Population_Dec2021']=""
df_main2=pd.DataFrame()
for (state,pop) in pop_pairs:
    df_main['Population_Dec2021'].loc[df_main['State']==state]=pop

df_main['Residents.Initiated.Rate']=df_main['Residents.Initiated']/df_main['Population_Dec2021']


df=df_main
df_main['Residents.Initiated.Rate']

,Residents.Initiated.Rate
54,0.04765
55,0.064774
56,0.095191
57,0.107883
58,0.153072
...,...
7314,0.797292
7315,0.801324
7316,0.796346
7317,0.796346


In [114]:
## Here we are reading in a file that was scrapped from the internet and previous papers about prioritization fo incarcerated by state
file_grouping=open('Data/Original/Prioritization.csv')
df_groupings=pd.read_csv(file_grouping)
df_groupings.drop('UCLA notes', axis=1, inplace=True)
df_groupings.drop('Matches with CPP', axis=1, inplace=True)
df_main['Prioritization?']=""
df_main['Date_prioritized']=""
df_main['Phase_state']=""
df_4=pd.DataFrame()
for (state,prioritized,prior_date,phase) in df_groupings.values:
    print(state,prioritized,prior_date,phase)
    df_main['Prioritization?'].loc[df_main['State']==state]=prioritized
    df_main['Date_prioritized'].loc[df_main['State']==state]=prior_date
    df_main['Phase_state'].loc[df_main['State']==state]=phase
df_main


Alabama yes 4/1/2021 1B
Alaska yes 2/1/2021 1B
Arizona yes 3/1/2021 1C
California yes 3/1/2021 1B
Colorado no nan nan
Connecticut yes nan nan
Delaware yes 4/1/2021 1C
Federal nan nan nan
Georgia no nan nan
Idaho yes 3/1/2021 1C
Kansas yes 1/1/2021 2
Maine no nan nan
Maryland yes 4/1/2021 2
Massachusetts yes 1/1/2021 1
Michigan no nan nan
Minnesota yes 3/1/2021 1B
Missouri no nan nan
New Hampshire no nan nan
New Jersey yes 12/1/2020 1A
North Carolina yes 3/1/2021 4
North Dakota yes 2/1/2021 1B
Ohio no nan nan
Pennsylvania yes 4/1/2021 1B
South Carolina yes 3/1/2021 1B
Virginia yes 1/1/2021 1B
Washington yes 3/1/2021 1B
West Virginia no nan nan
Wisconsin yes 3/1/2021 1C


,Date,State,Residents.Initiated,Month,Year,Population_Dec2021,Residents.Initiated.Rate,Prioritization?,Date_prioritized,Phase_state
54,2021-04-04,Alabama,871.0,4,2021,18279.0,0.04765,yes,4/1/2021,1B
55,2021-04-11,Alabama,1184.0,4,2021,18279.0,0.064774,yes,4/1/2021,1B
56,2021-04-18,Alabama,1740.0,4,2021,18279.0,0.095191,yes,4/1/2021,1B
57,2021-04-25,Alabama,1972.0,4,2021,18279.0,0.107883,yes,4/1/2021,1B
58,2021-05-02,Alabama,2798.0,5,2021,18279.0,0.153072,yes,4/1/2021,1B
...,...,...,...,...,...,...,...,...,...,...
7314,2022-06-05,Wisconsin,16016.0,6,2022,20088.0,0.797292,yes,3/1/2021,1C
7315,2022-06-12,Wisconsin,16097.0,6,2022,20088.0,0.801324,yes,3/1/2021,1C
7316,2022-06-19,Wisconsin,15997.0,6,2022,20088.0,0.796346,yes,3/1/2021,1C
7317,2022-06-26,Wisconsin,15997.0,6,2022,20088.0,0.796346,yes,3/1/2021,1C


In [108]:
## For this date, Wisconsin recorded a resident initiated population(211510) larger than the incarcerated population of the state (20088)

WI_problem_dates=['2022-01-23']

df_wi=df.loc[df['State']=='Wisconsin']

df_wi_probs=df_wi.loc[df_wi['Date'].isin(WI_problem_dates)]

df.drop(df_wi_probs.index, axis=0,inplace=True)

In [109]:
##Removing Arizona
states=['Alabama', 'Alaska', 'California', 'Colorado',
       'Connecticut', 'Delaware', 'Federal', 'Georgia', 'Idaho', 'Kansas',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Missouri', 'New Hampshire', 'New Jersey', 'North Carolina',
       'North Dakota', 'Ohio', 'Pennsylvania', 'South Carolina',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin']

df=df.loc[df['State'].isin(states)]


In [110]:
## Function to make rate data monotonic by state
def make_monotonic(df, cols=None):
    if cols is None:
        cols = df.columns

    df1 = df.copy()[cols]

    while True:
        mon_inc = (df1.diff().fillna(0) >= 0).all(axis=1)
        if mon_inc.all():
            break
        df1 = df1[mon_inc]
    return df1

In [111]:
## Adding data with corrected start date for states that lacked it, with information from news articles
state_real_start_zip=[('Michigan','2021-03-22'),('New Jersey','2020-12-15'),('Arizona','2021-03-05'),('Kansas','2021-02-06'),('Massachusetts','2021-01-18'),('Pennsylvania','2021-04-05')]
df_corrected_date=pd.DataFrame(state_real_start_zip,columns=['State','Start_Date'])
df_corrected_date['Start_Date']=pd.to_datetime(df_corrected_date.Start_Date, format='%Y-%m-%d')
weird_start_states=list(df_corrected_date['State'].unique())
df['Date_prioritized']=pd.to_datetime(df.Date_prioritized, format='%m/%d/%Y')

df_concat=pd.DataFrame()

In [112]:
# Cycling through states to make data monotonic and also include column regarding days in vaccination process adn days since last data entry.
for state in df['State'].unique():
    df_state=df.loc[df['State']==state]
    df_state['Mono.Initiated.Coverage']=make_monotonic(df_state,cols=['Residents.Initiated.Rate'])


    df_state=df_state.sort_values("Date")
    df_state=df_state.reset_index()


    if state in weird_start_states:
        new_start=list(df_corrected_date.loc[df_corrected_date['State']==state,'Start_Date'])
        start_date=new_start[0]
    else: start_date=df_state['Date'][0]
    df_state['Days']=(df_state['Date']-start_date).dt.days
    df_state['Days']= pd.to_numeric(df_state['Days'])
    df_state['Days_since_last']=df_state['Days']-df_state['Days'].shift(1)
    ##Because data does not include paired population data, we had to use population data from December 2021, this means that for some datees Virginia reports vaccination higher than the total population of the state,
    ## to correct for this we mask any coverage rates greater than 1 as 1 and mask any population values as teh total population of of the state
    if state=='Virginia':

        population=list(df_state['Residents.Initiated'].unique())[0]
        df_state["Mono.Initiated.Coverage"].mask( df_state["Mono.Initiated.Coverage"] > 1, 1 , inplace=True )
        df_state['Residents.Initiated'].mask( df_state['Residents.Initiated'] > population, population , inplace=True )
    df_concat= pd.concat([df_state,df_concat])
df_final=df_concat
df_final.to_csv('Data/Cleaned/jurisdiction_temporal.csv')